In [2]:
from openai import OpenAI
import time

class LegalNVIDIAAnswerer:
    def __init__(self, api_key, model="nvidia/llama-3.1-nemotron-70b-instruct"):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://integrate.api.nvidia.com/v1"
        )
        self.model = model

    def build_prompt(self, query, context_docs):
        context = "\n\n".join([f"Context {i+1}:\n{doc.strip()}" for i, doc in enumerate(context_docs)])
        prompt = (
            f"{context}\n\n"
            f"Question: {query.strip()}\n\n"
            f"Answer with a complete response followed by a short reasoning.\n"
            f"Format:\nAnswer: <your answer>\nReasoning: <your reasoning>"
        )
        return prompt

    def generate(self, query, context_docs, debug=False, retries=3, wait_time=10):
        prompt = self.build_prompt(query, context_docs)
        messages = [{"role": "user", "content": prompt}]
        
        for attempt in range(retries):
            try:
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    temperature=0.3,
                    top_p=1.0,
                    max_tokens=1024,
                    stream=False
                )
                response_text = completion.choices[0].message.content.strip()

                if debug:
                    print("🧠 Raw Response:\n", response_text)

                # Parse output
                answer = ""
                reasoning = ""
                if "Answer:" in response_text:
                    answer_part = response_text.split("Answer:", 1)[-1]
                    if "Reasoning:" in answer_part:
                        answer, reasoning = answer_part.split("Reasoning:", 1)
                    else:
                        answer = answer_part
                else:
                    answer = response_text

                return {
                    "answer": answer.strip(),
                    "reasoning": reasoning.strip() if reasoning else "(No explicit reasoning found)"
                }

            except Exception as e:
                print(f"⚠️ Attempt {attempt+1} failed: {e}")
                if attempt < retries - 1:
                    print(f"⏳ Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    raise RuntimeError("❌ All retries failed for NVIDIA LLM")

In [4]:
query = "Can police search a vehicle without a warrant under the Fourth Amendment?"
context_docs = [
    "In Carroll v. United States, the Court ruled that warrantless vehicle searches are permitted with probable cause.",
    "The 'automobile exception' is justified by the inherent mobility of vehicles and lower expectation of privacy.",
    "The scope of the search must be limited to areas where the evidence might reasonably be found.",
    "Probable cause remains a prerequisite — mere suspicion is not sufficient."
]

llm = LegalNVIDIAAnswerer(api_key="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")
response = llm.generate(query, context_docs, debug=True)

print("\n✅ Answer:", response["answer"])
print("📚 Reasoning:", response["reasoning"])

🧠 Raw Response:
 **Answer:** Yes, under certain conditions.
**Reasoning:** This answer is based on the integration of information from Contexts 1, 2, and 4. Specifically, according to Carroll v. United States (Context 1), warrantless vehicle searches are allowed with probable cause, not mere suspicion (Context 4). The "automobile exception" (Context 2) provides the legal justification for this, highlighting the vehicle's mobility and the lower privacy expectation as key factors.

✅ Answer: ** Yes, under certain conditions.
**
📚 Reasoning: ** This answer is based on the integration of information from Contexts 1, 2, and 4. Specifically, according to Carroll v. United States (Context 1), warrantless vehicle searches are allowed with probable cause, not mere suspicion (Context 4). The "automobile exception" (Context 2) provides the legal justification for this, highlighting the vehicle's mobility and the lower privacy expectation as key factors.
